# Running a 5-Point Calibration Scan With OSO

This notebook adapts the Five Point Calibration Scan notebook to work with PSI. Running this notebook requires that a PSI namespace be spun up with `DISH_LMC_ENABLED` set to true.

This notebook can be Run with BITE or SPFRx, with BITE requiring the optional BITE notebook to be run. 

###  For using BITE

 BITE Functionality has been spun out to a separate notebook, which should be referred to when needed. These sections are labeled as BITE STEP.

## 1 Setup 

### 1.1 Environment Setup

Start by importing all the libraries needed to run this notebook, including local ones.

In [1]:
import re
import sys

sys.path.append("../../src")

import json
import os
from time import sleep

from IPython.display import clear_output
from ska_oso_pdm import SBDefinition
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions import pdm_transforms
from ska_oso_scripting.functions.sb import create_sbi, load_sbd
from ska_oso_scripting.objects import SubArray, Telescope
from tango import DevFailed, DeviceProxy

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [2]:
!kubectl get ns | grep ska-mid-psi 

ci-ska-mid-psi-1659637082-alexschell                  Active   64m
ci-ska-mid-psi-1659637082-alexschell-sdp              Active   64m
ci-ska-mid-psi-1659741225-maecst                      Active   21m
ci-ska-mid-psi-1659741225-maecst-sdp                  Active   21m
ci-ska-mid-psi-1660558031-amjoshi                     Active   4h55m
ci-ska-mid-psi-1660558031-amjoshi-sdp                 Active   4h55m


And load it into the variables for the notebook:

In [3]:
namespace = "ci-ska-mid-psi-1659637082-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [4]  # assign boards
scheduling_block_definition = "adr-99-sb-test.json"  # SB def file in data/scheduling_block to use
server = "ska-sdp-kafka." + namespace + ".svc.cluster.local:9092"
subarray_id = 1

With the namespace set, use these links to monitor the system as the notebook is worked through

In [4]:
print("Monitor signal received by the vis pod:")
print(f"https://142.73.34.170/{namespace}/signal/display/")
print("Dashboard for monitoring device status:")
print(f"https://142.73.34.170/{namespace}/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run")
print("Dashboard for monitoring TMC status:")
print(f"https://142.73.34.170/{namespace}/taranta/dashboard?id=669ea8e32bc4790019e64bba&mode=run")
print("PgAdmin interface to monitor the databases and ensure SB/EB are properly set in them")
print(f"https://142.73.34.170/{namespace}/pgadmin4/browser/")

Monitor signal received by the vis pod:
https://142.73.34.170/ci-ska-mid-psi-1659637082-alexschell/signal/display/
Dashboard for monitoring device status:
https://142.73.34.170/ci-ska-mid-psi-1659637082-alexschell/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run
Dashboard for monitoring TMC status:
https://142.73.34.170/ci-ska-mid-psi-1659637082-alexschell/taranta/dashboard?id=669ea8e32bc4790019e64bba&mode=run
PgAdmin interface to monitor the databases and ensure SB/EB are properly set in them
https://142.73.34.170/ci-ska-mid-psi-1659637082-alexschell/pgadmin4/browser/


Get the SKUID pod:

In [5]:
!kubectl get svc -n $namespace | grep skuid
!kubectl get svc -n $namespace | grep kafka

ska-ser-skuid-test-1659637082-svc                ClusterIP      10.96.15.61      <none>            9870/TCP                                          62m
ska-sdp-kafka                                    ClusterIP      10.101.124.36    <none>            9092/TCP                                          62m
ska-sdp-kafka-headless                           ClusterIP      None             <none>            9092/TCP,9094/TCP,9093/TCP                        62m


And set the id value based on the name of this pod:

In [6]:
skuid_id = "ska-ser-skuid-test-1659637082-svc"
kafka_id = "ska-sdp-kafka"

As a work around step for SKUID pod access, get the IP from the pod, get the IP address of the SKUID pod then set that as our access url:

In [7]:
skuid_name = !kubectl -n $namespace get pods | grep ska-ser-skuid-test
skuid_name = skuid_name[0].split()[0]
print(skuid_name)
extracted_ip = !kubectl -n $namespace describe pod $skuid_name | grep ips -A 1
ip = re.search(r"[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}", str(extracted_ip))
print(ip.group(0))

kafka_name = !kubectl -n $namespace get pods | grep ska-sdp-kafka
kafka_name = kafka_name[0].split()[0]
print(kafka_name)
extracted_kafka_ip = !kubectl -n $namespace describe pod $kafka_name | grep ips -A 1
kafka_ip = re.search(r"[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}", str(extracted_kafka_ip))
print(kafka_ip.group(0))

ska-ser-skuid-test-1659637082-5f94c7fd96-pgprx


10.10.212.200
ska-sdp-kafka-0
10.10.253.150


Copy the IP from this store it to be written later:

In [8]:
SKUID_POD = ip.group(0) + ":9870"
KAFKA_POD = kafka_ip.group(0) + ":9092"

Next, load all the other vars the notebook will use. These should not need to be changed for this run.

In [9]:
# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"
# Don't use for now...
# SKUID_POD = skuid_id + "." + namespace + ".svc.cluster.local:5004"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf")
HW_CONFIG = os.path.join(DATA_DIR, "mid_telescope/cbf/hw_config")
SB_DIR = os.path.join(DATA_DIR, "scheduling_block")

# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

Next, set the environment arg for TANGO HOST, SKUID_POD and KAFKA, along with the oda API service endpoint:

In [10]:
os.environ["TANGO_HOST"] = TANGO_HOST
os.environ["SKUID_URL"] = SKUID_POD
os.environ["ODA_URL"] = f"http://142.73.34.170/{namespace}/oda/api/v6/"
os.environ["telescope"] = "mid"
print("SKUID service set to: ", os.environ["SKUID_URL"])
print("TANGO_HOST set to:", os.environ["TANGO_HOST"])
print("ODA endpoint set to:{}".format(os.environ["ODA_URL"]))
print("This can be checked via {}ui".format(os.environ["ODA_URL"]))

SKUID service set to:  10.10.212.200:9870
TANGO_HOST set to: databaseds-tango-base.ci-ska-mid-psi-1659637082-alexschell.svc.cluster.local:10000
ODA endpoint set to:http://142.73.34.170/ci-ska-mid-psi-1659637082-alexschell/oda/api/v6/
This can be checked via http://142.73.34.170/ci-ska-mid-psi-1659637082-alexschell/oda/api/v6/ui


With all the file paths defined the JSON files can be loaded in and checked.

In [11]:
print("Getting files...")

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"
SB_FILE = os.path.join(SB_DIR, scheduling_block_definition)

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    DISH_CONFIG_FILE,
    SB_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

Getting files...
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/mid_telescope/cbf/sys_params/load_dish_config.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/scheduling_block/adr-99-sb-test.json exists: ✔️


Next, set up the hw config to match the boards in use:

In [12]:
if target_boards_list[0] >= 11:
    print("Using HW Config File for Talons 11-14")
    hw_config = "hw_config_psi_11_14.yaml"
elif target_boards_list[0] >= 9:
    print("Using HW Config File for Talons 9, 10, 15, 16")
    hw_config = "hw_config_psi_9_10_15_16.yaml"
elif target_boards_list[0] >= 5:
    print("Using HW Config File for Talons 5-8")
    hw_config = "hw_config_psi_5_8.yaml"
elif target_boards_list[0] >= 1:
    print("Using HW Config File for Talons 1-4")
    hw_config = "hw_config_psi_1_4.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, hw_config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

target_boards_list = list(map(lambda x: x - (((x - 1) // 4) * 4), target_boards_list))
receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

Using HW Config File for Talons 1-4
HW config: ✔️


If the HW config file exists, it is loaded into the pod:

In [13]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml 

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Set Variables in the BITE Notebook</h3>
    If running the BITE notebook, at this point ensure the variables in step 1.2 are set, and match the ones set here.
</div>
</body>

### 1.3 Create Device Proxies

With the pod spun up, create device proxies to the devices used and check the connection to them.

In [14]:
# CSP Devices
csp_controller = DeviceProxy("mid-csp/control/0")
print(f"CSP Controller: {csp_controller.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
print(f"Central Node: {tmc_central_node.Status()}")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")
print(f"TMC subarray Node: {tmc_subarray.Status()}")

# For checking leaf node
leaf_node_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

# Deployer for setup and BITE for data mocking
deployer = DeviceProxy("mid_csp_cbf/ec/deployer")

# CBF Device
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
print(f"CBF subarray: {cbf_subarray.Status()}")

CSP Controller: The device is in DISABLE state.
Central Node: The device is in ON state.
TMC subarray Node: The device is in ON state.
CBF subarray: The device is in DISABLE state.


### 1.4 Downloading Requirements via the Deployer

First, set the board to deploy to and turn on the deployer device.

In [16]:
deployer.targetTalons = target_boards_list
print("Deployer will target board:", deployer.targetTalons)
deployer.generate_config_jsons()

Deployer will target board: [4]


Once started and configured, the required devices can then be downloaded.

In [17]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [18]:
deployer.configure_db()

### 1.5 Set up Execution Block

The first step is to generate an empty Execution Block, using the helper scripts to work with the API endpoint:

In [19]:
exec_block_id = oda_helper.create_eb(telescope="ska_mid")
print(exec_block_id)

eb-t0001-20250207-00002


## 2 Preparing Telescope

Start by setting the admin modes to allow for control of the telescope:

In [20]:
csp_controller.cbfSimulationMode = simulation_mode
csp_controller.commandTimeout = 99
sleep(2)
csp_controller.adminMode = 0
tmc_central_node.adminMode = 0

Ensure that these are set correctly, admin mode should be 0 (online) and cbfSimulationMode should be FALSE unless running in sim mode is desired.

In [21]:
print(csp_controller.adminMode)
print(tmc_central_node.adminMode)
print(csp_controller.cbfSimulationMode)

adminMode.ONLINE
adminMode.ONLINE
False


Start by loading in the VCC config (this step may need to be re-run):

In [22]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")

dish_config_json file contents: 
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}


In [23]:
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
# Wait for dishvcc to be loaded...
wait_seconds = 0
while not tmc_central_node.isDishVccConfigSet:
    clear_output(wait=True)
    print(f"Waiting for DishVCC to be set, {wait_seconds} seconds elapsed...")
    sleep(2)
    wait_seconds += 2

clear_output(wait=True)
print("DishVCC has been set!")

DishVCC has been set!


In [28]:
print(tmc_central_node.isDishVccConfigSet)

True


In [27]:
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{leaf_node_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{leaf_node_master.sourceDishVccConfig}"
)

TMC CSP Master's Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}

TMC CSP Master's Source Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"], "tm_data_filepath": "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"}


<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Load In BITE Config Data</h3>
    Now, if required, the BITE configuration data can be loaded in using the BITE notebook.
</div>
</body>

Set up the objects for controlling the telescope and subarray via OSO:

In [29]:
sub = SubArray(subarray_id)
tel = Telescope()

Now, while monitoring the Taranta dashboard, send the ON command to the telescope and monitor that the LURs Controller and subarrays come on (occasionally the ON command may not finish when everything has started):

In [30]:
tel.on()

1|2025-02-07T17:38:17.848Z|INFO|MainThread|_call_and_wait_for_transition|common.py#138||Using pub/sub to track telescopeState of ska_mid/tm_central/central_node
1|2025-02-07T17:38:17.876Z|INFO|MainThread|execute|tango_executor.py#242||Executing command: <Command('ska_mid/tm_central/central_node', 'TelescopeOn', )>
1|2025-02-07T17:38:17.884Z|INFO|MainThread|wait_for_transition|common.py#329||Waiting for telescopeState to transition to ON
1|2025-02-07T17:38:17.884Z|INFO|MainThread|read_event|tango_executor.py#185||Reading event from the queue with timeout None
1|2025-02-07T17:38:18.003Z|INFO|MainThread|read_event|tango_executor.py#185||Reading event from the queue with timeout None
1|2025-02-07T17:38:20.803Z|INFO|MainThread|wait_for_transition|common.py#345||telescopeState reached target state ON


<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Generate BITE Data</h3>
    With the telescope on, section 3 of the BITE notebook can be run, to generate the BITE data that will be used during the LSTV replay.
</div>
</body>

In [31]:
print(f"CSP controller state: {csp_controller.State()}")

CSP controller state: ON


### 2.1 Define Scheduling Block

Temporary work around for ingress issues:

In [32]:
os.environ["SKUID_URL"] = SKUID_POD
print("SKUID service set to: ", os.environ["SKUID_URL"])

SKUID service set to:  10.10.212.200:9870


Now, use the SKUID service to generate processing and execution block IDs from the scheduling block definition.

In [33]:
sb = load_sbd(SB_FILE)

In [34]:
# Configure sb items for current board:
sb.dish_allocations.dish_ids = receptor_ids
# for now run with 1 fsp
sb.csp_configurations[0].midcbf.subbands[0].correlation_spws[0].receptors = receptor_ids

In [35]:
sbi = create_sbi(sb)

1|2025-02-07T17:41:48.861Z|INFO|MainThread|create_sbi|sb.py#28||New SBI ID mapping: sbi-mvp01-20200325-00001 -> sbi-t0001-20250207-00003


In [36]:
print(sb)
print(sbi)

interface='https://schema.skao.int/ska-oso-pdm-sbd/0.1' sbd_id='sbi-mvp01-20200325-00001' name=None description=None telescope=TelescopeType.SKA_MID metadata=Metadata(version=1, created_by='Alex Schell', created_on=datetime.datetime(2025, 2, 5, 15, 43, 53, 971548, tzinfo=TzInfo(UTC)), last_modified_by='Alex Schell', last_modified_on=datetime.datetime(2025, 2, 5, 15, 43, 53, 971548, tzinfo=TzInfo(UTC)), pdm_version='17.0.1') prj_ref=None activities={'observe': GitScript(function_args={'init': PythonArguments(args=[], kwargs={'subarray_id': 1}), 'main': PythonArguments(args=[], kwargs={})}, kind=ScriptKind.GIT, repo='https://gitlab.com/ska-telescope/oso/ska-oso-scripting', path='git://scripts/test_example_mid_script.py', branch='master', commit=None)} targets=[Target(target_id='Hercules A', name='', pointing_pattern=PointingPattern(active=PointingKind.FIVE_POINT, parameters=[FivePointParameters(kind=PointingKind.FIVE_POINT, offset_arcsec=5.0)]), reference_coordinate=EquatorialCoordinates

Using this, create the JSON for the scheduling block, subbing in the Kafka location, and validating it.

In [38]:
sb_json = SBDefinition.model_dump_json(sbi)
# sb_json = sb_json.replace("<KAFKA_HOST>", KAFKA_POD)

In [37]:
sbi = SBDefinition.model_validate_json(sb_json)

In [39]:
print(json.dumps(json.JSONDecoder().decode(sb_json), indent=2))

{
  "interface": "https://schema.skao.int/ska-oso-pdm-sbd/0.1",
  "sbd_id": "sbi-t0001-20250207-00003",
  "telescope": "ska_mid",
  "metadata": {
    "version": 1,
    "created_by": "Alex Schell",
    "created_on": "2025-02-05T15:43:53.971548Z",
    "last_modified_by": "Alex Schell",
    "last_modified_on": "2025-02-05T15:43:53.971548Z",
    "pdm_version": "17.0.1"
  },
  "activities": {
    "observe": {
      "function_args": {
        "init": {
          "kwargs": {
            "subarray_id": 1
          }
        },
        "main": {}
      },
      "kind": "git",
      "repo": "https://gitlab.com/ska-telescope/oso/ska-oso-scripting",
      "path": "git://scripts/test_example_mid_script.py",
      "branch": "master"
    }
  },
  "targets": [
    {
      "target_id": "Hercules A",
      "name": "",
      "pointing_pattern": {
        "active": "FivePointParameters",
        "parameters": [
          {
            "kind": "FivePointParameters",
            "offset_arcsec": 5.0
       

## 3 Running the Observation

### 3.1 Assigning Resources

Now, the resources can be assigned to the telescope, either via getting the resources from the scheduling block:

In [ ]:
assign_requests = pdm_transforms.create_cdm_assign_resources_request_from_scheduling_block(
    sub.id, sbi
)

1|2025-02-07T17:42:04.453Z|INFO|MainThread|create_mid_assign_resources_request|wrapper.py#85||Setting dish : frozenset({'SKA100'}) 
1|2025-02-07T17:42:04.455Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp_create.py#150||Setting target attribute -> target_id:Hercules A , reference_coordinate:<EquatorialCoordinates: (16h51m07.98866878s +04d59m35.54713147s ICRS)> 
1|2025-02-07T17:42:04.456Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp_create.py#167||Resulting SDP coordinate for targetid:Hercules A -> system:ICRF3 , ra:[252.78328611990997] , dec:[4.99320753652]
1|2025-02-07T17:42:04.463Z|INFO|MainThread|create_mid_assign_resources_request|wrapper.py#94||Setting SDP configuration for EB: eb-t0001-20250207-00002 


In [ ]:
assign_requests.sdp_config.processing_blocks[0].parameters["transport_protocol"] = "udp"
assign_requests.sdp_config.processing_blocks[0].parameters["use_network_definition"] = True
assign_requests.sdp_config.processing_blocks[0].parameters["dry_run"].remove
assign_requests.sdp_config.processing_blocks[0].parameters["extra_helm_values"]["receiver"][
    "options"
]["reception"]["reset_time_indexing_after_each_scan"] = True

KeyError: 'extra_helm_values'

In [55]:
print(json.dumps(json.JSONDecoder().decode(assign_requests.model_dump_json()), indent=2))

{
  "subarray_id": 1,
  "dish": {
    "receptor_ids": [
      "SKA100"
    ]
  },
  "sdp_config": {
    "interface": "https://schema.skao.int/ska-sdp-assignres/0.4",
    "execution_block": {
      "eb_id": "eb-t0001-20250207-00002",
      "max_length": 1000.0,
      "context": {},
      "beams": [
        {
          "beam_id": "vis0",
          "function": "visibilities"
        }
      ],
      "channels": [
        {
          "channels_id": "vis_channels",
          "spectral_windows": [
            {
              "count": 14880,
              "start": 0,
              "stride": 1,
              "freq_min": 350013440.0,
              "freq_max": 550000640.0,
              "spectral_window_id": "vis_spw_1"
            }
          ]
        }
      ],
      "polarisations": [
        {
          "polarisations_id": "all",
          "corr_type": [
            "XX",
            "XY",
            "YX",
            "YY"
          ]
        }
      ],
      "fields": [
        {
        

In [ ]:
stripped_assign_requests = assign_requests
stripped_assign_requests.interface = ""
stripped_assign_requests.dish.receptor_ids = receptor_ids
print(stripped_assign_requests)

Now assign the resources to the subarray.

In [42]:
sub.assign_from_cdm(assign_requests, timeout=1000)

1|2025-02-07T17:42:19.617Z|INFO|MainThread|_call_and_wait_for_transition|common.py#138||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2025-02-07T17:42:19.628Z|INFO|MainThread|_get_id_from_params_or_generate_new_id|transactions.py#172||Generated transaction ID txn-t0001-20250207-000000006
1|2025-02-07T17:42:19.629Z|INFO|MainThread|log_entry|transactions.py#132||Transaction[txn-t0001-20250207-000000006]: Enter[AssignResources] with parameters [{}] marker[01479]
1|2025-02-07T17:42:19.629Z|INFO|MainThread|_call_and_wait_for_transition|common.py#169||Added transaction_id to command: <Command('ska_mid/tm_central/central_node', 'AssignResources', '{"subarray_id": 1, "dish": {"receptor_ids": ["SKA100"]}, "sdp": {"interface": "https://schema.skao.int/ska-sdp-assignres/0.4", "execution_block": {"eb_id": "eb-t0001-20250207-00002", "max_length": 1000.0, "context": {}, "beams": [{"beam_id": "vis0", "function": "visibilities"}], "channels": [{"channels_id": "vis_channels", "spectra

Check that the visibility pod spins up correctly, with the signal displays.

In [45]:
!kubectl -n $namespace-sdp get pods | grep vis-receive

NAME                                                          READY   STATUS    RESTARTS   AGE
proc-pb-t0001-20250207-00004-script-r5x79                     1/1     Running   0          4m15s
proc-pb-t0001-20250207-00005-pointing-offset-87998687-ndt6w   1/1     Running   0          3m48s
proc-pb-t0001-20250207-00005-script-tx7gc                     1/1     Running   0          4m14s


<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Starting LSTV Replay</h3>
    If using BITE for data stream generation, the additional BITE notebook can be run at this point.
</div>
</body>

### 3.3 Run the Scan(s)

To start running the scans (or just one) first create the configuration request from the scheduling block:

In [46]:
scan_def_to_config_req_mapping = pdm_transforms.create_cdm_configure_request_from_scheduling_block(
    sbi
)

KeyError: 'M83'

#### Running a Single Scan

For a single scan, load in part of the scan configuration just for that run, and send the configure commands to the subarray. Once this is complete, run the scan command itself. Using the scheduling block sets how long the scan will run, so it will finish by itself.

In [ ]:
print(scan_def_to_config_req_mapping)
config_req = scan_def_to_config_req_mapping["scan-definition-28007"]
print(config_req)
sub.configure_from_cdm(config_req[0], timeout=500)
sleep(5)
sub.scan()

#### Running Multiple Scans

To run multiple scans, iterate over the scan sequence, and use the ids in it to define a request and then send it to the subarray.

In [ ]:
print("...")
print(scan_def_to_config_req_mapping)
print("...")
for scan_def_id in sb.scan_sequence:
    config_req = scan_def_to_config_req_mapping[scan_def_id]
    print(config_req)
    print("--------------")
    for request in config_req:
        sub.configure_from_cdm(request, timeout=500)
        sleep(2)
        sub.scan()

## 4 Cleanup

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Stopping LSTV Replay</h3>
    Now with the initial (or only) scan done, if using BITE, run step 5 in the BITE notebook to stop the LSTV Replay.
</div>
</body>

Once the scans are done, clear out the configuration from the subarray by sending the END command, followed by releasing the resources it used:

In [ ]:
sub.end()
sleep(10)

In [ ]:
sub.release()

The telescope can now be turned off.

In [ ]:
tel.off()